<a href="https://colab.research.google.com/github/nasif-raihan/pose-estimation/blob/main/pose_estimation_with_detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configure

In [ ]:
!pip install -qqU torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html

!pip install -qq cython pyyaml==5.1

!pip install -qqU 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!pip install -qq detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

     |████████████████████████████████| 703.8 MB 13 kB/s 
     |████████████████████████████████| 6.6 MB 61.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.5.0+cu101 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.5.0+cu101 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.5.0+cu101 which is incompatible.
fastai 2.7.10 requires torchvision>=0.8.2, but you have torchvision 0.6.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 14.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.5.0+cu101 which is 

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.99  # set threshold for this model

# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml")

predictor = DefaultPredictor(cfg)

model_final_5ad38f.pkl: 491MB [00:20, 24.1MB/s]                           


# Run Model

**17 key points from coco dataset**


'nose': 0,

'left_eye': 1, 'left_ear': 3, 'left_shoulder': 5, 'left_elbow': 7 'left_wrist': 9, 'left_hip': 11, 'left_knee': 13, 'left_ankle': 15,

'right_eye': 2, 'right_ear': 4, 'right_shoulder': 6, 'right_elbow': 8 'right_wrist': 10, 'right_hip': 12, 'right_knee': 14, 'right_ankle': 16

In [ ]:
!gdown -q # put the gid here

In [ ]:
cap = cv2.VideoCapture('/content/gait_raw.mp4')
fps = cap.get((cv2.CAP_PROP_FPS))
print(fps)

29.97002997002997


In [ ]:
success, img = cap.read()
w, h, c = img.shape
video = cv2.VideoWriter("/content/rendered-video.mp4", cv2.VideoWriter_fourcc('m','p','4','v'), fps, (h, w))

In [ ]:
outputs = predictor(img)

print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)
print(outputs['instances'].pred_keypoints.shape)

tensor([0], device='cuda:0')
Boxes(tensor([[  1.2405,  21.2492, 101.2059, 220.9573]], device='cuda:0'))
torch.Size([1, 17, 3])


In [ ]:
skeleton = {
    (0, 1):   'm',
    (0, 2):   'c',
    (1, 3):   'm',
    (2, 4):   'c',
    (0, 5):   'm',
    (0, 6):   'c',
    (5, 7):   'm',
    (7, 9):   'm',
    (6, 8):   'c',
    (8, 10):  'c',
    (5, 6):   'y',
    (5, 11):  'm',
    (6, 12):  'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def take_all_keyjoints(data, instance_no, axis):
  name = []

  for i in range(data.shape[1]):
    name.append(data[instance_no][i][axis])

  return name

In [ ]:
while success:
  success, img = cap.read()

  if success == False:
    break

  outputs = predictor(img)

  poses = outputs['instances'].pred_keypoints.cpu()

  instance_x = take_all_keyjoints(poses, 0, 0)
  instance_y = take_all_keyjoints(poses, 0, 1)

  for edge, color in skeleton.items():
    p1, p2 = edge

    if color == 'm':
        COLOR = (255, 0, 255)
    elif color == 'c':
        COLOR = (0, 255, 255)
    else:
        COLOR = (255,255,0)

    x1 = poses[0][p1][0]
    y1 = poses[0][p1][1]

    x2 = poses[0][p2][0]
    y2 = poses[0][p2][1]
    
    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), COLOR, 2)

  video.write(img)

video.release()